LDGM codes
---


Given a source $y\in\{0,1\}^n$ and a so-called Low-Density Generator Matrix (LDGM) $G\in\{0,1\}^{n,k}, k<n$, find $x\in\{0,1\}^k$ such that $d_H(Gx,s)$ is minimal.
Following the notation in \cite{castanheira2010lossy}, we assign indices $i\in 1,\dots,k$ to variables and $a\in 1,\dots,n$ to factors.
The compression rate is given by $R=k/n$.

After the usual change of variables $\sigma_i=(-1)^{x_i}$, $s_a=(-1)^{y_a}$, one has
\begin{equation}
    \begin{aligned}
    \min_x \sum_{a=1}^n (Gx)_a\oplus y_a =&
    \min_x \sum_{a=1}^n \left(\bigoplus_{i\in\partial a}x_i\right)\oplus y_a \\
    =& \min_\sigma \left[-\sum_{a=1}^ns_a\prod_{i\in\partial a}\sigma_i\right]\\
    =& \min_\sigma \mathcal{H}(\sigma)
    \end{aligned}
\end{equation}
The corresponding Boltzmann distribution
\begin{equation}
    P(\sigma) = \frac1Z \prod_{a=1}^n e^{\beta s_a \prod\limits_{i\in\partial a}\sigma_i}
\end{equation}

Max-Sum equations
---

\begin{equation}
\boxed{
    \begin{aligned}
    h_{ia} =& \sum_{b\in\partial i \setminus a} u_{bi}\\
    u_{ai} =& {\rm sign}\left(s_a\prod_{j\in\partial a \setminus i}\sigma_j\right)\min\left\{1,\min_{j\in\partial a \setminus i}|h_{ja}|\right\}
    \end{aligned}
    }
\end{equation}
Notice that the second equation implies $|u_{ai}|\le1$ !


In [1]:
function iter_factor(q, k)
    p[0] = 1-(1-q[0])^k
    p[1] = p[-1] = 0.5*(1-p[0])
    p
end

iter_factor (generic function with 1 method)

In [2]:
function convolve(p1, p2, maxd)
    q = fill(0.0,-maxd:maxd)
    for f1 ∈ eachindex(p1)
        for f2 ∈ eachindex(p2)
            q[clamp(f1+f2, -maxd, maxd)] += p1[f1]*p2[f2]
        end
    end
    q
end


function iter_variable(p,d, maxd)
    q = fill(0.0, -maxd:maxd); q[0]=1
    for i in 1:d
        q = convolve(q,p, maxd)
    end
    q ./= sum(q)
    
end

iter_variable (generic function with 1 method)

In [3]:
function iter_slow_factor(q, k)
    N = lastindex(q)
    p = fill(0.0, -1:1)
    for s in (-1,1)
        for fs ∈ Iterators.product(fill(-N:N,k)...)
            f = min(1, minimum(abs.(fs)))*sign(prod(fs)*s)
            p[f] += prod(q[f1] for f1 ∈ fs)
        end
    end
    p ./= sum(p)
end


iter_slow_factor (generic function with 1 method)

In [9]:
function iter_slow_var(p, d) 
    N = lastindex(p)
    q = fill(0.0, -d:d)
    for us in Iterators.product(fill(-1:1,d)...)
        h = sum(us)
        q[h] += prod(p[u] for u ∈ us)
    end
    q ./= sum(q)
end

iter_slow_var (generic function with 1 method)

In [10]:
using OffsetArrays
p = OffsetArray([0.4, 0.2, 0.4], -1:1)
iter_slow_var(p, 3)

7-element OffsetArray(::Vector{Float64}, -3:3) with eltype Float64 with indices -3:3:
 0.064
 0.096
 0.24
 0.19999999999999998
 0.24
 0.096
 0.064

In [11]:
iter_variable(p,3, 5)

11-element OffsetArray(::Vector{Float64}, -5:5) with eltype Float64 with indices -5:5:
 0.0
 0.0
 0.06399999999999999
 0.09599999999999999
 0.23999999999999996
 0.19999999999999998
 0.23999999999999994
 0.09599999999999999
 0.06399999999999999
 0.0
 0.0

In [12]:
q = OffsetArray([0.1, 0.05, 0.7, 0.05, 0.1], -2:2)

5-element OffsetArray(::Vector{Float64}, -2:2) with eltype Float64 with indices -2:2:
 0.1
 0.05
 0.7
 0.05
 0.1

In [13]:
iter_slow_factor(q, 3)

3-element OffsetArray(::Vector{Float64}, -1:1) with eltype Float64 with indices -1:1:
 0.013500000000000024
 0.9730000000000001
 0.013500000000000024

In [14]:
iter_factor(q,3)

3-element OffsetArray(::Vector{Float64}, -1:1) with eltype Float64 with indices -1:1:
 0.013500000000000012
 0.973
 0.013500000000000012

RS over Max-Sum
---
We have, for fixed degrees

\begin{equation}
    \begin{aligned}
p_k\left(u\right)= &\sum_{h_{1},\dots,h_{k}}\sum_s\frac12\delta\left(u,\min\left\{1,\min_{j=1,\dots,k}\left|h_{j}\right|\right\}\text{sign}\left(s\prod_{j=1}^{k}h_{j}\right)\right)\prod_{j=1}^{k}q\left(h_{j}\right)\\
q_d\left(h\right)=&\sum_{u_{1},\dots,u_{d}} \delta\left(h,\sum_{a=1}^{d} u_a \right)\prod_{a=1}^{d} p(u_a)
\end{aligned}
\end{equation}
and averaging over the degree profile,
\begin{equation}
 \begin{aligned}
    p(u) &\propto \sum_k k K_k p_{k-1}(u)\\
q(h) &\propto \sum_d d \Lambda_d q_{d-1}(h)
\end{aligned}
\end{equation}

Following the same steps as in the case of LDPC codes and using the fact that $|u|\le1$,

\begin{equation}
   \boxed{
   \begin{aligned}
   p_k(u) =& p_0\delta_0+(1-p_0)\frac{\delta_{+1}+\delta_{-1}}{2},\quad p_0=1-\left(1-q(0)\right)^k
%   \begin{cases}
%   1 - \left[1-q(0)\right]^{k} & u=0\\
%   \frac{\left[1-q(0)\right]^{k}}{2} & u=1
%   \end{cases}
   \\
   q_d(h) =& p\underbrace{ \circledast\dots\circledast}_{d \text{ times}}p
   \end{aligned}}
\end{equation}


In [26]:
function RS(Pk, Λ; N=100, tol=1e-5, maxiter=100, damp=0.9, maxd=lastindex(Λ))
    ks = [k for k in eachindex(Pk) if Pk[k] > tol]
    ds = [d for d in eachindex(Λ) if Λ[d] > tol]
    @assert sum(Pk[ks]) ≈ 1 && sum(Λ[ds]) ≈ 1
    
    R =  sum(k*Pk[k] for k=ks) / sum(d*Λ[d] for d=ds)

    p = fill(1.0, -1:1); p ./= sum(p)
    q = fill(0.0, -maxd:maxd) 
    for iter=1:maxiter
        q1 = sum(d*Λ[d]*iter_variable(p, d-1, maxd) for d=ds)
        q1 ./= sum(q1)
        @show q1
        p1 = sum(k*Pk[k]*iter_factor(q, k-1) for k=ks)
        p1 ./= sum(p1)
        @show p1
        err = min(maximum(abs, p1 - p),maximum(abs, q1 - q)) ; err < tol && (@show err iter; break)
        q .= q .* damp .+ q1 .* (1-damp)
        p .= p .* damp .+ p1 .* (1-damp)
    end
    p, q, R
end

RS (generic function with 1 method)

In [25]:
tol = 1e-5
ks = [k for k in eachindex(Pk) if Pk[k] > tol]
ds = [d for d in eachindex(Λ) if Λ[d] > tol]
maxd=lastindex(Λ)
p = fill(1.0, -1:1); p ./= sum(p)
q = fill(0.0, -maxd:maxd) 
q1 = sum(d*Λ[d]*iter_variable(p, d-1, maxd) for d=ds)

7-element OffsetArray(::Vector{Float64}, -3:3) with eltype Float64 with indices -3:3:
 0.0
 0.09999999999999998
 0.6666666666666665
 0.7666666666666666
 0.6666666666666665
 0.09999999999999998
 0.0

In [27]:
Λ = OffsetArray([0, 0, 0.7, 0.3], 0:3)
Pk = OffsetArray([0, 0, 1, 0], 0:3)
RS(Pk, Λ) 

q1 = [0.0, 0.04347826086956521, 0.28985507246376807, 0.3333333333333333, 0.28985507246376807, 0.04347826086956521, 0.0]
p1 = [0.5, 0.0, 0.5]
q1 = [0.0, 0.04793478260869564, 0.2952173913043478, 0.31369565217391304, 0.2952173913043478, 0.04793478260869564, 0.0]
p1 = [0.48333333333333334, 0.033333333333333326, 0.48333333333333334]
q1 = [0.0, 0.051656521739130434, 0.29888115942028987, 0.2989246376811594, 0.29888115942028987, 0.051656521739130434, 0.0]
p1 = [0.46931521739130433, 0.06136956521739134, 0.46931521739130433]
q1 = [0.0, 0.054714045502229386, 0.3013963699759596, 0.28777916904362216, 0.3013963699759596, 0.054714045502229386, 0.0]
p1 = [0.45743746376811595, 0.0851250724637681, 0.45743746376811595]
q1 = [0.0, 0.05718506719661016, 0.3031305012198605, 0.27936886316705867, 0.3031305012198605, 0.05718506719661016, 0.0]
p1 = [0.44730475893912325, 0.1053904821217535, 0.44730475893912325]
q1 = [0.0, 0.05914694125563363, 0.3043296335746667, 0.27304685033939935, 0.3043296335746667, 0.05914694

q1 = [0.0, 0.050654076457156826, 0.2979619795261583, 0.30276788803336985, 0.2979619795261583, 0.050654076457156826, 0.0]
p1 = [0.35334286779215796, 0.2933142644156841, 0.35334286779215796]
q1 = [0.0, 0.05047268573403518, 0.29779046070642307, 0.30347370711908356, 0.29779046070642307, 0.05047268573403518, 0.0]
p1 = [0.3528701866112736, 0.29425962677745277, 0.3528701866112736]
q1 = [0.0, 0.050296448925204865, 0.2976222700625347, 0.30416256202452085, 0.2976222700625347, 0.050296448925204865, 0.0]
p1 = [0.3524094825941921, 0.29518103481161584, 0.3524094825941921]
q1 = [0.0, 0.05012519411140346, 0.29745736727624894, 0.30483487722469516, 0.29745736727624894, 0.05012519411140346, 0.0]
p1 = [0.35196040623354685, 0.2960791875329063, 0.35196040623354685]
q1 = [0.0, 0.049958755899959614, 0.29729571009537503, 0.30549106800933074, 0.29729571009537503, 0.049958755899959614, 0.0]
p1 = [0.3515226217489574, 0.29695475650208525, 0.3515226217489574]
q1 = [0.0, 0.04979697521635148, 0.2971372546601412, 0.30

([0.3424294238230766, 0.31514115235384704, 0.3424294238230766], [0.0, 0.04662154672209148, 0.2937219577112751, 0.3192864297343794, 0.2937219577112751, 0.04662154672209148, 0.0], 0.8695652173913044)